In [1]:
# Importing libraries
import pandas as pd
from sqlalchemy import create_engine

### Success Criteria
1. Match the KPI start and end date of the availed service, payout should be evaluated based on the KPI dates.
2. Personal Loan will have only one payin-payout event based on the percentage criteria of the decided KPI.
3. If the account disbursed date or disbursed amount was not captured properly in MIS, then these cases will be excluded from the final output.
4. DateOfSale will be equivalent to LoanDisbursedDate.
\
\
Status == "Success", SubStatus == "Loan Disbursed"

In [1]:
def personal_loan(mis, rules, leads):
    try:
        # Analyzing the paying_payout_rules for saving accounts
        rules = (
            rules
            .query('ProductType == "Personal Loan"')
        )

        # Creating a SQLAlchemy Engine object to connect to an in-memory SQLite database
        engine = create_engine('sqlite:///:memory:')
        # Writing DataFrame to the SQLite database
        rules.to_sql('rules', engine, index=False)
        mis.to_sql('mis', engine, index=False)
        leads.to_sql('leads', engine, index=False)

        # Calculating the payin-payout for MIS
        query1 = '''
        SELECT m.*,
            ROUND((r.KPI1PayinPercentage / 100.0) * m.DisbursedAmount, 0) AS KPI1PayinAmount,
            ROUND((r.KPI1PayoutPercentage / 100.0) * m.DisbursedAmount, 0) AS KPI1PayoutAmount,
            0 AS KPI2PayinAmount,
            0 AS KPI2PayoutAmount
        FROM mis AS m
            JOIN rules AS r
                ON m.LoanDisbursedDate
                BETWEEN r.KPIStartDate AND r.KPIEndDate
        WHERE m.SourceType = r.SourceType
            AND m.MediumType = r.MediumType
            AND m.ProductType = r.ProductType
        '''
        success_events = pd.read_sql_query(query1, engine)

        success_events.to_sql('success_events', engine, index=False)

        # Generating the final upload sheet
        query2 = '''
        SELECT CURRENT_TIMESTAMP AS CreatedAt,
            s.SourceType,
            s.MediumType,
            s.ProductType,
            ml.SPId,
            s.LeadId,
            'Success' AS Status,
            'Loan Disbursed' AS SubStatus,
            date(LoanDisbursedDate) AS DateOfSale,
            date(CURRENT_TIMESTAMP) AS DateOfRevenue,
            s.KPI1PayinAmount,
            s.KPI1PayoutAmount,
            s.KPI2PayinAmount,
            s.KPI2PayoutAmount
        FROM success_events AS s
            JOIN leads AS ml
                ON ml.LeadId = s.LeadId
        WHERE ml.Status != 'Success'
            '''
        final_result = pd.read_sql_query(query2, engine)

        final_result['TotalPayin'] = final_result['KPI1PayinAmount'] + final_result['KPI2PayinAmount']
        final_result['TotalPayout'] = final_result['KPI1PayoutAmount'] + final_result['KPI2PayoutAmount']

        return final_result
        pass

    except Exception as e:
        return str(e)